In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut,GeocoderUnavailable
from pandas.core.common import flatten
import ast
from tqdm import tqdm
import time
import pickle


In [ ]:
full_year = pd.read_csv("../../input-data/temp-data.json")

In [ ]:
full_year

In [ ]:
full_year.columns

In [ ]:
full_year_essential = full_year[["_id", "section_name", "keywords", "pub_date"]].copy()

In [ ]:
full_year_essential['keywords'] = full_year_essential['keywords'].apply(ast.literal_eval)

In [ ]:
full_year_essential = full_year_essential.explode("keywords")

In [ ]:
full_year_essential["keyword_type"] = full_year_essential["keywords"].get("name")

In [ ]:
full_year_essential["keyword"] = full_year_essential["keywords"].get("value")

In [ ]:
full_year_essential["keyword_type"] = full_year_essential["keywords"].apply(lambda x: x.get("name") if isinstance(x, dict) else None)
full_year_essential["keyword"] = full_year_essential["keywords"].apply(lambda x: x.get("value") if isinstance(x, dict) else None)
full_year_essential["keyword_rank"] = full_year_essential["keywords"].apply(lambda x: x.get("rank") if isinstance(x, dict) else None)

In [ ]:
locations = full_year_essential[full_year_essential["keyword_type"] == "glocations"]

In [ ]:
locations = locations["keyword"].unique()

In [ ]:
locations

In [ ]:
places_df = pd.DataFrame(locations, columns=["location"])

In [ ]:
places_df

In [ ]:
places_df['country'] = places_df['location'].apply(lambda x: x[x.find("(")+1:x.find(")")] if "(" in x and ")" in x else x)

In [ ]:
places_df

In [ ]:
geolocator = Nominatim(user_agent="geo_locator")

In [ ]:
location_countries = {}

In [ ]:
# Function to get country
def get_country(location):
    try:
        geo = geolocator.geocode(location, exactly_one=True, language='en', addressdetails=False)
        if geo:
            return geo.address.split(",")[-1].strip()
        else:
            return "Not found"
    except GeocoderTimedOut:
        return "Timeout"
    except GeocoderUnavailable:
        return "Unavailable"

In [ ]:
places_df['retrieved_country'] = places_df.apply(lambda x: get_country(x["country"]), axis=1)

In [ ]:
places_df.to_csv("../../input-data/nyt_retrived_countries.csv")

In [ ]:
data = []
for place, country in places_df.items():
    place_coverage = full_year_essential[full_year_essential["keyword"] == place]
    ids_of_articles = place_coverage["_id"].values.tolist()
    print(f"Place: {place}, Country: {country}")
    data.append((place, country, len(place_coverage), ids_of_articles))


In [ ]:
places_df = pd.DataFrame(data, columns=["place_keyword", "country", "count_of_articles", "ids_of_articles"])

In [ ]:
places_df= places_df.sort_values("country")

In [ ]:
places_df.to_csv("../../data/places/coverage_by_location.csv")